# CSE 6240 - Web Search & Text Mining - Homework 3

In [1]:
# __author__ = 'Bhanu Verma'
# GTid = '903151012'

In [72]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import os, os.path
from scipy.sparse import lil_matrix,linalg
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from scipy.sparse.linalg import norm
from sklearn.cross_validation import train_test_split
from scipy.stats import pearsonr
%matplotlib inline

In [3]:
def get_matrix_dimensions():
    doc_size = 0
    vocab_size = 0
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count==0:
                doc_size = int(line)
            elif count==1:
                vocab_size = int(line)
            else:
                break
                
            count+=1
    
    f.close()
    
    return (doc_size,vocab_size)

In [4]:
def populate_matrix():
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count > 2:
                arr = line.split()
                i = int(arr[0])-1
                j = int(arr[1])-1
                val = int(arr[2])
                mat[i,j] = val
            count+=1
    f.close()

In [5]:
def get_triplet_list():
    triplet_list = []
    ordered_list = []
    for i in range(n_doc):
        # randomly generate 6 numbers, 1 extra to take care of the case where random number id is equal to i
        rand_arr = np.random.choice(n_doc,6,replace=False)
        doc_list = [] # list of indices of documents
        corr_val = [] # list of calculated correlation coefficients
        tuple_list = []
        
        # fill up correlation values
        for j in range(len(rand_arr)):
            if i!=j:
                a = get_dense_row(i)
                b = get_dense_row(rand_arr[j])
                pear_val = pearsonr(a,b) # it returns a tuple where first element is the coefficient value
                corr_val.append(pear_val[0])
                doc_list.append(rand_arr[j])
                if len(corr_val) == 5:
                    break

        # build up triplets
        for a in range(len(corr_val)):
            for b in range(a+1,len(corr_val)):
                if b < len(corr_val):
                    if corr_val[a] >= corr_val[b]:
                        triplet_list.append((i,doc_list[a],doc_list[b]))
                        tuple_list.append((i,doc_list[a],doc_list[b]))
                    else:
                        triplet_list.append((i,doc_list[b],doc_list[a]))
                        tuple_list.append((i,doc_list[b],doc_list[a]))
        
    return triplet_list

In [6]:
def get_dense_row(index):
    return dense_mat[index].tolist()[0] # fix for the issue, tolist() here returns matrix of rowsize 1

In [7]:
def get_sparse_row(index):
    return mat[index]

In [8]:
def dense(sparse):
    sparse = sparse.todense()
    return sparse[0].tolist()[0]

# Question 2.a

In [9]:
# get matrix dimensions
filepath = 'Data/docword.kos.txt'
size_tup = get_matrix_dimensions()

In [10]:
n_doc = size_tup[0]
n_vocab = size_tup[1]

In [11]:
# populate lil_matrix dimensions
mat = lil_matrix((size_tup[0],size_tup[1]),dtype=np.int32)
populate_matrix()

# convert lil_matrix to csr matrix for faster computation
mat = mat.tocsr()
dense_mat = mat.todense()

In [12]:
triplets = get_triplet_list()

In [126]:
# split triplet array into training and testing data
train,test = train_test_split(triplets, train_size = 0.8, random_state=42)

# Question 1 - OASIS Implementation

In [127]:
w = identity(n_vocab,format='csr')
training_loss_list = []
testing_loss_list = []
c = 0.1

In [128]:
def get_similarity_score(a,w_mat,b):
    result = a*w_mat*b
    return result[0,0]

In [129]:
def get_loss(a_i,a_plus,a_neg,w_mat):
    res_a = get_similarity_score(a_i,w_mat,a_plus.transpose())
    res_b = get_similarity_score(a_i,w_mat,a_neg.transpose())
    val = 1-res_a+res_b
    return max(0,val)

In [ ]:
for i in range(len(train)):
    p_i = mat[train[i][0]]
    p_plus = mat[train[i][1]]
    p_neg = mat[train[i][2]]
    
    loss = get_loss(p_i,p_plus,p_neg,w)
    if (i%200 == 0):
        print "number of iterations - ", i
    
    if loss != 0:
        diff = (p_plus - p_neg).transpose()
        ones = np.ones((n_vocab,n_vocab))
        diff_csr = csr_matrix(diff.multiply(ones))
        elements = csr_matrix(p_i.multiply(ones))
        v_i = elements.multiply(diff_csr).transpose()
        norm_val = norm(v_i, 'fro')**2 
        tau = min(c,loss/norm_val)
        w = w + tau*v_i

    train_loss = 0
    test_loss = 0
    
    if (i%200 == 0):
        # calculate Training Loss
        for z in range(len(train)):
            a_i = mat[train[z][0]]
            a_plus = mat[train[z][1]]
            a_neg = mat[train[z][2]]
            train_loss += get_loss(a_i,a_plus,a_neg,w)
        training_loss_list.append(train_loss)
        
        # calculate Testing Loss
        for k in range(len(test)):
            t_i = mat[test[k][0]]
            t_plus = mat[test[k][1]]
            t_neg = mat[test[k][2]]
            test_loss += get_loss(t_i,t_plus,t_neg,w)
        testing_loss_list.append(test_loss)
        
        print "Training Error", train_loss
        print "Testing Error", test_loss
        print "----------------------"

number of iterations -  0
Training Error

In [ ]:
a = csr_matrix([[1,0,3],[1,3,4],[1,0,7]])
b = 2*a
print b.toarray()

In [ ]:
a = mat[train[i][1]]